## Imports

In [1]:
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

from autokeras import StructuredDataRegressor

In [2]:
# Load dataset

boston = load_boston()

df = pd.DataFrame(boston.data)

df.columns = boston.feature_names
df['PRICE'] = boston.target 

print(df.shape)

(506, 14)


In [3]:
X = df.drop('PRICE', axis=1).values
y = df.PRICE.values

In [4]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(339, 13) (167, 13) (339,) (167,)


In [5]:
# Search
search = StructuredDataRegressor(max_trials=30, loss='mean_squared_error')

# Perform the search
search.fit(x=X_train, y=y_train, verbose=1)

Trial 30 Complete [00h 00m 05s]
val_loss: 471.8982849121094

Best val_loss So Far: 7.226643085479736
Total elapsed time: 00h 21m 11s
INFO:tensorflow:Oracle triggered exit
Epoch 1/63
11/11 [==============================] - 1s 5ms/step - loss: 571.4692 - mean_squared_error: 571.4692
Epoch 2/63
11/11 [==============================] - 0s 5ms/step - loss: 460.6294 - mean_squared_error: 460.6294
Epoch 3/63
11/11 [==============================] - 0s 4ms/step - loss: 388.6752 - mean_squared_error: 388.6752
Epoch 4/63
11/11 [==============================] - 0s 4ms/step - loss: 331.3669 - mean_squared_error: 331.3669
Epoch 5/63
11/11 [==============================] - 0s 4ms/step - loss: 282.8588 - mean_squared_error: 282.8588
Epoch 6/63
11/11 [==============================] - 0s 4ms/step - loss: 240.0187 - mean_squared_error: 240.0187
Epoch 7/63
11/11 [==============================] - 0s 4ms/step - loss: 202.0583 - mean_squared_error: 202.0583
Epoch 8/63
11/11 [===========================

In [6]:
# evaluate the model
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print(f'MAE: {mae}')

MAE: 11.781134605407715


In [7]:
# Get the best performing model
model = search.export_model()

# Summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 13)                0         
_________________________________________________________________
normalization (Normalization (None, 13)                27        
_________________________________________________________________
dense (Dense)                (None, 32)                448       
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              33792 

In [8]:
if not os.path.isdir('model'):
    os.mkdir('model')

model.save('model/boston.h5')

NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Got a model or layer MultiCategoryEncoding with weights [<tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000001BE4DCC27F0>, <tensorflow.python.keras.engine.base_layer_utils.TrackableWeightHandler object at 0x000001BE4DCC2E80>]